## 1. Tải dữ liệu

In [73]:
!wget --no-check-certificate \
    https://storage.googleapis.com/protonx-cloud-storage/data.txt
data = open('data.txt').read()

--2024-01-19 11:23:34--  https://storage.googleapis.com/protonx-cloud-storage/data.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.2.207, 2607:f8b0:4023:c0d::cf
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.2.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93578 (91K) [text/plain]
Saving to: ‘data.txt.4’

data.txt.4          100%[===================>]  91.38K  --.-KB/s    in 0.001s  

2024-01-19 11:23:35 (114 MB/s) - ‘data.txt.4’ saved [93578/93578]



## 2. Thêm các thư viện cần thiết

In [74]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import tensorflow.keras.utils as ku
import numpy as np

## 3. Xử lý dữ liệu

In [75]:
corpus = data.lower().split("\n")

In [76]:
corpus

['from fairest creatures we desire increase,',
 "that thereby beauty's rose might never die,",
 'but as the riper should by time decease,',
 'his tender heir might bear his memory:',
 'but thou, contracted to thine own bright eyes,',
 "feed'st thy light'st flame with self-substantial fuel,",
 'making a famine where abundance lies,',
 'thyself thy foe, to thy sweet self too cruel.',
 "thou that art now the world's fresh ornament",
 'and only herald to the gaudy spring,',
 'within thine own bud buriest thy content',
 'and, tender churl, makest waste in niggarding.',
 'pity the world, or else this glutton be,',
 "to eat the world's due, by the grave and thee.",
 'when forty winters shall beseige thy brow,',
 "and dig deep trenches in thy beauty's field,",
 "thy youth's proud livery, so gazed on now,",
 "will be a tatter'd weed, of small worth held:",
 "then being ask'd where all thy beauty lies,",
 'where all the treasure of thy lusty days,',
 'to say, within thine own deep-sunken eyes,',

In [77]:
['from fairest creatures we desire increase, <end>',
 "that thereby beauty's rose might never die, <end>",
 'but as the riper should by time decease, <end>',
 'his tender heir might bear his memory: <end>']

['from fairest creatures we desire increase, <end>',
 "that thereby beauty's rose might never die, <end>",
 'but as the riper should by time decease, <end>',
 'his tender heir might bear his memory: <end>']

In [78]:
corpus[:4]

['from fairest creatures we desire increase,',
 "that thereby beauty's rose might never die,",
 'but as the riper should by time decease,',
 'his tender heir might bear his memory:']

### 3.1. Xây dựng tokenizer

In [79]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

In [80]:
total_words

3211

### 3.2. Tách câu

Tách từng câu thành nhiều phần có chiều dài tăng dần để làm từng điểm dữ liệu

In [81]:
input_sequences = []
for line in corpus:
  token_list = tokenizer.texts_to_sequences([line])[0]
  for i in range(1, len(token_list)):
    n_gram_sequence = token_list[:i+1]
    input_sequences.append(n_gram_sequence)

In [82]:
input_sequences[:10]

[[34, 417],
 [34, 417, 877],
 [34, 417, 877, 166],
 [34, 417, 877, 166, 213],
 [34, 417, 877, 166, 213, 517],
 [8, 878],
 [8, 878, 134],
 [8, 878, 134, 351],
 [8, 878, 134, 351, 102],
 [8, 878, 134, 351, 102, 156]]

In [83]:
tokenizer.sequences_to_texts([[34, 417]])

['from fairest']

Thực chất các câu sẽ được cắt thành nhiều phần như thế này

In [84]:
for point in input_sequences[:10]:
  print(" ".join(tokenizer.sequences_to_texts([point])))

from fairest
from fairest creatures
from fairest creatures we
from fairest creatures we desire
from fairest creatures we desire increase
that thereby
that thereby beauty's
that thereby beauty's rose
that thereby beauty's rose might
that thereby beauty's rose might never


### 3.3. Chia features, label

Thực hiện Padding

In [85]:
# pad sequences
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

Các câu được tách bên trên sẽ được padding với việc chèn các giá trị 0 vào trước câu để tạo ra các câu có chiều dài bằng nhau.

Việc **padding vào trước** vì bài toán này ta sẽ sinh từ từ phía sau nên thông tin bên phải là những giá trị khác 0.

In [86]:
input_sequences[:10]

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,  34, 417],
       [  0,   0,   0,   0,   0,   0,   0,   0,  34, 417, 877],
       [  0,   0,   0,   0,   0,   0,   0,  34, 417, 877, 166],
       [  0,   0,   0,   0,   0,   0,  34, 417, 877, 166, 213],
       [  0,   0,   0,   0,   0,  34, 417, 877, 166, 213, 517],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   8, 878],
       [  0,   0,   0,   0,   0,   0,   0,   0,   8, 878, 134],
       [  0,   0,   0,   0,   0,   0,   0,   8, 878, 134, 351],
       [  0,   0,   0,   0,   0,   0,   8, 878, 134, 351, 102],
       [  0,   0,   0,   0,   0,   8, 878, 134, 351, 102, 156]],
      dtype=int32)

Cắt cột cuối cùng của ma trận bên trên để làm nhãn

In [87]:
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
# Chuyển thành One Hot vector


In [88]:
label[i:i+1]

array([351], dtype=int32)

In [89]:
for i in range(10):
  print("{} ---> {}".format(" ".join(tokenizer.sequences_to_texts(predictors[i:i+1])), tokenizer.sequences_to_texts([label[i:i+1]])[0]))

from ---> fairest
from fairest ---> creatures
from fairest creatures ---> we
from fairest creatures we ---> desire
from fairest creatures we desire ---> increase
that ---> thereby
that thereby ---> beauty's
that thereby beauty's ---> rose
that thereby beauty's rose ---> might
that thereby beauty's rose might ---> never


Chuyển từng nhãn thành vector one hot với số lượng là số từ trong từ điển

In [90]:
label = ku.to_categorical(label, num_classes=total_words)

In [91]:
label

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

## 4. Xây dựng model

Mô hình này có nhiệm vụ học khả năng một từ hiện tại có xác suất xuất hiện sau một số lượng từ nhất định như thế nào.

Mạng bao gồm:

- 1 lớp Embedding với chiều embedding là 100
- Một lớp 1 Bidirectional với cell LSTM 150 node
- Một lớp LSTM với 100 node
- Mạng nơ ron phân loại gồm một lớp ẩn


In [92]:
model = Sequential()

model.add(Embedding(total_words, 100, input_length=max_sequence_len-1)) # biến thành embedding

model.add(Bidirectional(LSTM(150, return_sequences = True))) # đưa qua LSTM với cả chiều trái qua phải và phải qua trái

model.add(Dropout(0.2))

model.add(LSTM(100))

model.add(Dense(total_words/2, activation='relu', kernel_regularizer=regularizers.l2(0.01)))

model.add(Dense(total_words, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 10, 100)           321100    
                                                                 
 bidirectional_2 (Bidirecti  (None, 10, 300)           301200    
 onal)                                                           
                                                                 
 dropout_2 (Dropout)         (None, 10, 300)           0         
                                                                 
 lstm_5 (LSTM)               (None, 100)               160400    
                                                                 
 dense_4 (Dense)             (None, 1605)              162105    
                                                                 
 dense_5 (Dense)             (None, 3211)              5156866   
                                                      

Tiến hành training

In [93]:
 history = model.fit(predictors, label, epochs=130, verbose=1)

Epoch 1/130
484/484 [==============================] - 16s 21ms/step - loss: 6.9119 - accuracy: 0.0209
Epoch 2/130
484/484 [==============================] - 6s 13ms/step - loss: 6.5005 - accuracy: 0.0224
Epoch 3/130
484/484 [==============================] - 5s 10ms/step - loss: 6.3902 - accuracy: 0.0251
Epoch 4/130
484/484 [==============================] - 5s 11ms/step - loss: 6.2723 - accuracy: 0.0288
Epoch 5/130
484/484 [==============================] - 6s 12ms/step - loss: 6.1809 - accuracy: 0.0352
Epoch 6/130
484/484 [==============================] - 5s 10ms/step - loss: 6.1042 - accuracy: 0.0391
Epoch 7/130
484/484 [==============================] - 6s 12ms/step - loss: 6.0269 - accuracy: 0.0396
Epoch 8/130
484/484 [==============================] - 5s 10ms/step - loss: 5.9359 - accuracy: 0.0426
Epoch 9/130
484/484 [==============================] - 5s 11ms/step - loss: 5.8297 - accuracy: 0.0501
Epoch 10/130
484/484 [==============================] - 6s 12ms/step - loss: 5.70

### 5. Dự đoán 10 từ tiếp theo

Câu mồi

In [94]:
test_seq = 'despite of wrinkles'

Từ câu mồi
- Dự đoán ra từ tiếp theo từ các từ của câu hiện tại
- Nối từ đã được dự đoán vào câu hiện tại
- Tiếp tục dùng câu hiện tại này để dự đoán
- Thực hiện đến khi chiều dài của câu đạt một giới hạn cụ thể hoặc số từ sinh ra đạt giới hạn

In [95]:
next_words = 10

for _ in range(next_words):
  # Chuyển câu thành vector
  token_list = tokenizer.texts_to_sequences([test_seq])[0]

  # Padding câu
  token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')

  # Dự đoán từ tiếp theo
  predicted = model.predict(token_list, verbose=0) # sinh ra 1 từ

  output_word = ""

  predicted_id = np.argmax(predicted)



  if predicted_id in tokenizer.index_word:
    output_word = tokenizer.index_word[predicted_id] #chuyển sang từ

    if output_word == '<end>':
        break
    test_seq += " " + output_word # nối vào câu để tiếp tục sinh từ khác
  else:
    break

print(test_seq)


despite of wrinkles this thy golden time do now stell'd halt miss'd near sit sit twain loving breast to give give give die wrong new spent sit keep dear moan ' torn mother go on small days another life do say ill ' decay die ' so press die deem'd bred free latch chary spent twain spent miss'd mother chary hate hate hate skill mother mother hits hits blushing glory prove stand him in worth alone bright days bright room out to owe outworn held held decays age make die forth hence prove lie by woe kind desire alone prove foes stand ill foes shown stand forth woe life do die appear men foes lie to thee in thee one dead gone bring die mind sun stand you spent shown stand new age and changes words of thee doth kings ride prove stand thee new dear face lack smells dear sight and ground stand new desire thee show new fired spent pride to morrow hast him with me more doth grow up kind lies near near sit broken sit sit twain halt sit hits mother hate hate hate esteem esteem new mother hits hits 

Để sinh tốt hơn bạn có thể tham khảo thêm thuật toán **Beam Search**

In [96]:
# despite of wrinkles
# this: 0.6
# that: 0.54

How old are you? As an AI language model, I do not have an age like humans do. I was created and trained by OpenAI and my knowledge is constantly being updated and improved.
How old are you? As -> an
How old are you? As an -> AI
How old are you? As an AI -> language
How old are you? As an AI language -> model

How old are you? As an AI language model, I do not have an age like humans do. I was created and trained by OpenAI and my knowledge is constantly being updated and improved. -> <end>


